In [19]:
import os
from tqdm import tqdm
from detector.detector import load_model_detector
import pandas as pd
import cv2

In [21]:
test_name2id = {'klikun':2, 'maliy': 1, 'shipun':3}

In [22]:
# df = pd.read_csv("../create_markup/test.csv")
df = pd.read_csv("../create_markup/train.csv")

In [23]:
path_detector_onnx = r"./model_data/best.onnx"

class_names = ['klikun', 'maliy', 'shipun']
# providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']
providers = ['CPUExecutionProvider']
detector_model = load_model_detector(
    model_path=path_detector_onnx,
    class_names=class_names,
    providers=providers,
    input_shape=(640, 640),
    score_thresh=0.05
)

In [26]:
predict_list = []
root_ds = r"C:\workspace\hakaton\dataset"
for path_img in tqdm(df.path_img.unique()):
    true_label = df[df.path_img == path_img]['type_name'].iloc[0]
    file_path = os.path.join(root_ds, path_img)
    if not os.path.exists(file_path):
        print('not exists', file_path)
        continue
    img = cv2.imread(file_path)[:,:,::-1]
    result = detector_model(img)
    for row in result:
        predict_list.append([*row, path_img, true_label])
predict_df = pd.DataFrame(predict_list, columns=['name', 'conf', 'x1', 'y1', 'x2', 'y2', 'path_img', 'true_label'])

 53%|█████▎    | 4222/8030 [04:10<03:10, 20.03it/s]

not exists C:\workspace\hakaton\dataset\maliy\images\Torgachkin=C58=3-DSC05379.jpg


 62%|██████▏   | 5015/8030 [04:54<02:33, 19.58it/s]

not exists C:\workspace\hakaton\dataset\maliy\images\Torgachkin=C58=3-DSC05516_.jpg


100%|██████████| 8030/8030 [08:17<00:00, 16.13it/s]


In [27]:
predict_df.to_csv('predict_train.csv', index=False)

In [30]:
predict_df.columns

Index(['name', 'conf', 'x1', 'y1', 'x2', 'y2', 'path_img', 'true_label'], dtype='object')

In [31]:

# Отбор name с максимальным coef внутри каждой группы path_img
result = predict_df.loc[predict_df.groupby('path_img')['conf'].idxmax()]
result


,name,conf,x1,y1,x2,y2,path_img,true_label
710,klikun,0.825610,934,730,640,640,klikun\images\1.jpg,klikun
3360,klikun,0.767366,46,17,299,222,klikun\images\1211999571.jpg,klikun
426,klikun,0.854084,59,44,191,224,klikun\images\1213000553.jpg,klikun
2350,klikun,0.943373,65,31,504,334,klikun\images\1227728011.jpg,klikun
2711,klikun,0.853998,106,10,520,383,klikun\images\1227740047.jpg,klikun
...,...,...,...,...,...,...,...,...
12157,shipun,0.478413,205,460,640,640,shipun\images\img_993.jpg,shipun
10032,shipun,0.956572,215,42,640,640,shipun\images\img_994.jpg,shipun
10947,shipun,0.948468,841,541,640,640,shipun\images\img_995.jpg,shipun
11287,shipun,0.896092,264,219,640,640,shipun\images\img_996.jpg,shipun


In [28]:
predict_df.groupby(by='path_img').

,conf,x1,y1,x2,y2
path_img,,,,,
klikun\images\1.jpg,0.825610,1706,872,640,640
klikun\images\1211999571.jpg,0.767366,46,17,299,222
klikun\images\1213000553.jpg,0.854084,137,46,255,224
klikun\images\1227728011.jpg,0.943373,498,217,600,336
klikun\images\1227740047.jpg,0.853998,106,10,520,383
...,...,...,...,...,...
shipun\images\img_993.jpg,0.478413,205,460,640,640
shipun\images\img_994.jpg,0.956572,215,42,640,640
shipun\images\img_995.jpg,0.948468,841,541,640,640


In [40]:
# acc = 0
# for path in df.path_img.unique():
#     res = predict_df[predict_df.path_img == path]
#     res = res.groupby('name').count()
#     res.sort_values('conf', ascending=False, inplace=True)
#     # print(res.index[0])
#     if len(res) and res.index[0] == df[df.path_img==path]['type_name'].iloc[0]:
#         acc += 1
#
# acc/len(df.path_img.unique())
# print('Максимальное количество.')


Максимальное количество.


In [41]:
import numpy

In [6]:
acc = 0
for path in tqdm(df.path_img.unique()):
    res = predict_df[predict_df.path_img == path]
    temp = res.groupby('name').count()
    temp = temp.sort_values('conf', ascending=False)
    cls_type = None
    if len(temp)>=2 and temp.conf.iloc[0] == temp.conf.iloc[1]:
        # если нет доминантного класса по количеству
        # самый уверенный бокс
        # res = predict_df[predict_df.path_img == path]
        res = res.sort_values('conf', ascending=False)
        cls_type = res['name'].iloc[0]
    elif len(temp):
        cls_type = temp.index[0]
    if cls_type == df[df.path_img==path]['type_name'].iloc[0]:
        acc += 1


print(f'accuracy: {acc/len(df.path_img.unique())}')

100%|██████████| 1899/1899 [00:02<00:00, 788.68it/s]

accuracy: 0.957345971563981


In [18]:
acc = 0
empty_count = 0
for path in tqdm(df.path_img.unique()):
    res = predict_df[predict_df.path_img == path]
    if len(res) == 0:
        empty_count += 1
        continue

    res['w'] = (res['x2'] - res['x1'])
    res['h'] = (res['y2'] - res['y1'])
    res['area'] = (res.w * res.h)

    temp = res.sort_values('area', ascending=False)
    cls_type = temp['name'].iloc[0]
    # print(cls_type)
    # break
    # temp = res.groupby('name').count()
    # temp = temp.sort_values('conf', ascending=False)
    # cls_type = None
    # if len(temp)>=2 and temp.conf.iloc[0] == temp.conf.iloc[1]:
    #     # если нет доминантного класса по количеству
    #     # самый уверенный бокс
    #     # res = predict_df[predict_df.path_img == path]
    #     res = res.sort_values('conf', ascending=False)
    #     cls_type = res['name'].iloc[0]
    # elif len(temp):
    #     cls_type = temp.index[0]
    if cls_type == df[df.path_img==path]['type_name'].iloc[0]:
        acc += 1

print(f'accuracy: {acc/len(df.path_img.unique())}')
print(f'Empty predict count = {empty_count}')

  0%|          | 0/1899 [00:00<?, ?it/s]C:\Users\annza\AppData\Local\Temp\ipykernel_20372\478804981.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['w'] = (res['x2'] - res['x1'])
C:\Users\annza\AppData\Local\Temp\ipykernel_20372\478804981.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['h'] = (res['y2'] - res['y1'])
C:\Users\annza\AppData\Local\Temp\ipykernel_20372\478804981.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

accuracy: 0.9473407056345445
Empty predict count = 6


In [42]:
predict_df

,name,conf,x1,y1,x2,y2,path_img
0,klikun,0.921919,136,83,478,457,klikun\images\2838665848.jpg
1,klikun,0.839110,19,4,396,483,klikun\images\1485516500.jpg
2,klikun,0.568135,221,73,448,345,klikun\images\1486047569.jpg
3,klikun,0.934844,91,119,449,322,klikun\images\1483387320.jpg
4,klikun,0.832700,19,143,640,640,klikun\images\4011766400.jpg
...,...,...,...,...,...,...,...
3949,shipun,0.938023,342,137,569,409,shipun\images\img_1321.jpg
3950,shipun,0.922765,362,219,640,640,shipun\images\img_2139.jpg
3951,shipun,0.775354,223,194,640,352,shipun\images\img_611.jpg
3952,shipun,0.964563,131,193,640,640,shipun\images\img_154.jpg


In [43]:
predict_df.to_csv('predict_test.csv', index=False)